In [1]:
#Here Model is loaded and train is rescaled and randomly placed and test is just scaled 
    

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)

train_datagen = ImageDataGenerator(
    rescale =1./255,
    rotation_range=40, 
    width_shift_range=0.2,     # Randomly shift images horizontally
    height_shift_range=0.2,    # Randomly shift images vertically
    shear_range=0.2,           # Random shear transformation
    zoom_range=0.2,            # Random zoom
    horizontal_flip=True,      # Random horizontal flip
    fill_mode='nearest'        # Fill mode for newly created pixels
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for test data

# Load the data
test_dir = "C:/Users/Kunal/OneDrive/Desktop/Pythom/Practical Example Regerssion/Alzhimer problem/Dataset/Combined Dataset/test"
train_dir = "C:/Users/Kunal/OneDrive/Desktop/Pythom/Practical Example Regerssion/Alzhimer problem/Dataset/Combined Dataset/train"


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
#Now We will build a CNN Model

In [13]:
# Train the model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
# Load pre-trained ResNet50 model (without top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Create the model by adding custom layers on top of ResNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x) 
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)  # 4 classes for Alzheimer's progression

# Final model
model_resnet = Model(inputs=base_model.input, outputs=x)

optimiser = Adam(learning_rate=0.0001)
# Compile the model
model_resnet.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['accuracy'])

# Print summary
model_resnet.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,694,084 (98.02 MB)

 Trainable params: 2,104,324 (8.03 MB)

 Non-trainable params: 23,589,760 (89.99 MB)

In [15]:
# Train the model
history = model_resnet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,  # Set number of epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


Epoch 1/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 821s 3s/step - accuracy: 0.4265 - loss: 1.2720 - val_accuracy: 0.1406 - val_loss: 4.1275
Epoch 2/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 876s 3s/step - accuracy: 0.4567 - loss: 1.2174 - val_accuracy: 0.5000 - val_loss: 3.1321
Epoch 3/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 873s 3s/step - accuracy: 0.4559 - loss: 1.2032 - val_accuracy: 0.5023 - val_loss: 1.0119
Epoch 4/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 863s 3s/step - accuracy: 0.4751 - loss: 1.1682 - val_accuracy: 0.5078 - val_loss: 3.2988
Epoch 5/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 856s 3s/step - accuracy: 0.4849 - loss: 1.1563 - val_accuracy: 0.1469 - val_loss: 3.6356
Epoch 6/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 831s 3s/step - accuracy: 0.4971 - loss: 1.1366 - val_accuracy: 0.2398 - val_loss: 1.7097
Epoch 7/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 837s 3s/step - accuracy: 0.4976 - loss: 1.1265 - val_accuracy: 0.1352 - val_loss: 2.6459
Epoch 8/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 843s 3s/step - accuracy: 0.5006 - loss: 1.1062 - val_accu